In [ ]:
from google.colab import drive#
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# packages to store and manipulate data
import pandas as pd
import numpy as np
 
# plotting packages
import matplotlib.pyplot as plt
import seaborn as sns
 
# model building package
import sklearn
 
# package to clean text
import re

# Topic modelling

In [ ]:
%%capture
!pip install pyLDAvis
import re, nltk, spacy, string

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint

#import pandas as pd
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from gensim.corpora.dictionary import Dictionary
from nltk.tokenize import word_tokenize
import gensim

In [ ]:
WNlemma = nltk.WordNetLemmatizer()
def lemmatize_stemming(message):
    return WNlemma.lemmatize(message)

In [ ]:
from gensim.parsing.preprocessing import STOPWORDS
all_stopwords_gensim = STOPWORDS.union(set(['nigeria', 'nigerian','protest','let','want',u'\'s']))

In [ ]:
def preprocess(message):
  result = [lemmatize_stemming(word) for word in gensim.utils.simple_preprocess(message) if not word in all_stopwords_gensim and len(word) > 2]
  return (" ").join(result)
  
data_in['processed_docs'] = data_in['cleaned_tweets'].map(preprocess)

In [ ]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=20,
                             max_df=0.9,                 # minimum required occurences of a word 
                             stop_words='english',             # remove stop words
                             #lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=5000,             # max number of unique words. Build a vocabulary that only consider the top max_features ordered by term frequency across the corpus
                            )

data_vectorized = vectorizer.fit_transform(data_in['processed_docs'])

lda_model = LatentDirichletAllocation(n_components=6, # Number of topics
                                      learning_method='online',
                                      random_state=0,       
                                      n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
3       1.465019 -47.625065       1        1  18.268039
2      54.854797 -36.476795       2        1  17.503116
5      32.212929 -61.444572       3        1  16.392021
0       5.111096 -14.110332       4        1  16.199167
4      26.350170 -33.374702       5        1  15.957455
1      38.109818  -7.222781       6        1  15.680202, topic_info=            Term          Freq         Total Category  logprob  loglift
1242         dey   8069.000000   8069.000000  Default  30.0000  30.0000
1917         god   7736.000000   7736.000000  Default  29.0000  29.0000
1945  government  11217.000000  11217.000000  Default  28.0000  28.0000
3486   protester  10115.000000  10115.000000  Default  27.0000  27.0000
3339      police  10362.000000  10362.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
2004         guy   1782.813632   4894.923603   Topic6  -4.7801   0.8428
4920        work   1302.644085   2496.231601   Topic6  -5.0939   1.2024
4822       watch   1178.594527   1576.731453   Topic6  -5.1940   1.5617
4494        time   1392.243535   7951.479134   Topic6  -5.0274   0.1103
4983       youth   1420.035618  10172.637643   Topic6  -5.0077  -0.1162

[288 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3         3  0.999438     abeg
20        4  0.999364    abuja
53        1  0.999435   action
103       3  0.999189   africa
104       4  0.999544  african
...     ...       ...      ...
4983      2  0.232781    youth
4983      3  0.003047    youth
4983      4  0.240842    youth
4983      5  0.383578    youth
4983      6  0.139590    youth

[321 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 6, 1, 5, 2])

In [ ]:
def print_top_words(model, vectorizer, n_top_words):
    feature_names = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        message = "\nTopic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [ ]:
print_top_words(lda_model, vectorizer, n_top_words=35)


Topic #0: god time voice tweet help support need youth people matter life protester movement abuja home till change mean bless news speak medium dear know hero soul real great money lie sad win shall best imagine

Topic #1: amp stop come live buhari president police retweet state stay look demand guy pray curfew shame soro soke officer won safe video heard military youth time army sure work proud watch shoot governor ooo station

Topic #2: protester police people right peaceful killing life need citizen new peace sars power nigerian innocent kill stand youth country join tired child shooting fighting justice force human protect come order cause soldier lost security reform

Topic #3: government end people good thing dont think sars better know way let thug morning need day thank happening like time said protesting die rest road focus tomorrow twitter hope president brutality care coming guy action

Topic #4: country leader youth government day generation fight love man nation like sta

In [ ]:
doc_topic = pd.DataFrame(lda_model.transform(data_vectorized),columns=['Media coverage','Activism','Police Brutality','No response from public figures','Questions about moving forward/past','Lekki Massacre'])
doc_topic.head(4)

,Media coverage,Activism,Police Brutality,No response from public figures,Questions about moving forward/past,Lekki Massacre
0,0.077743,0.116719,0.011133,0.133934,0.011111,0.649361
1,0.023854,0.023845,0.023849,0.023887,0.880621,0.023944
2,0.083333,0.083333,0.583333,0.083333,0.083333,0.083333
3,0.020833,0.021013,0.145829,0.020833,0.770658,0.020833


In [ ]:
data_with_topic = data_in_sel.join(doc_topic)
data_with_topic

In [ ]:
data_in_reset=data_in.reset_index(drop=True)

In [ ]:
top_data_with_topic = pd.DataFrame(data_with_topic.iloc[:,6:].columns.values[np.argsort(-data_with_topic.iloc[:,6:].values, axis=1)[:, :3]],
                  columns = ['1st Max','2nd Max','3rd Max'])
top_data_with_topic 

,1st Max,2nd Max,3rd Max
0,Lekki Massacre,No response from public figures,Activism
1,Questions about moving forward/past,Lekki Massacre,No response from public figures
2,Police Brutality,Activism,Media coverage
3,Questions about moving forward/past,Police Brutality,Activism
4,Activism,Media coverage,No response from public figures
...,...,...,...
204486,Questions about moving forward/past,Police Brutality,Activism
204487,Media coverage,Activism,Police Brutality
204488,Lekki Massacre,Activism,Media coverage
204489,Questions about moving forward/past,Police Brutality,Lekki Massacre


In [ ]:
data_in_sel = data_in_reset[['tweet_id','tweet_date','tweet_time_','tweet','cleaned_tweets','processed_docs']]

In [ ]:
data_with_topic['tweet'][3]

'@PoliceNG_CRU your officers are asking for war. This is robbery. Is it until you get featured on BBC and CNN disgracing the nation internationally that you will #endsars ?'

In [ ]:
final_topic = data_in_sel.join(top_data_with_topic)
final_topic

,tweet_id,tweet_date,tweet_time_,tweet,cleaned_tweets,processed_docs,1st Max,2nd Max,3rd Max
0,1312104957391237120.0,2020-10-02,18:59:34,The key to escape SARS is wearing your faded s...,the key to escape sars is wearing your faded s...,key escape sars wearing faded shirt jean rubbe...,Lekki Massacre,No response from public figures,Activism
1,1312270183487152128.0,2020-10-03,05:56:07,You see that #BlackLivesMatter go be like exam...,you see that go be like example when nigeria y...,like example youth finally turn extortion,Questions about moving forward/past,Lekki Massacre,No response from public figures
2,1312290451370762240.0,2020-10-03,07:16:39,I AM TIRED #EndSARS,i am tired,tired,Police Brutality,Activism,Media coverage
3,1312292483670462464.0,2020-10-03,07:24:43,@PoliceNG_CRU your officers are asking for war...,your officers are asking for war this is robbe...,officer asking war robbery featured bbc cnn di...,Questions about moving forward/past,Police Brutality,Activism
4,1312295604123467776.0,2020-10-03,07:37:07,I know some of the SARS guys from the State CI...,i know some of the sars guys from the state ci...,know sars guy state cid help dont drag try run...,Activism,Media coverage,No response from public figures
...,...,...,...,...,...,...,...,...,...
204486,1333580178371481600.0,2020-12-01,01:14:25,Exactly correct brother; the world's justice s...,exactly correct brother the world's justice sy...,exactly correct brother world justice rigged l...,Questions about moving forward/past,Police Brutality,Activism
204487,1333580639988187136.0,2020-12-01,01:16:15,Man❤️am🥰ashuga-tam😗🙏❤️🌹😇🎶#EndSARS,manamashuga-tam,manamashuga tam,Media coverage,Activism,Police Brutality
204488,1333581222505705472.0,2020-12-01,01:18:34,🙏Inshallah🙏🌹❤️Allah is wise. #EndSARS,inshallahallah is wise,inshallahallah wise,Lekki Massacre,Activism,Media coverage
204489,1333581627327320064.0,2020-12-01,01:20:11,@PK_Defend @F_BALOCHI1 @PTIofficial @ImranKhan...,allah knows all look in your hand and your hea...,allah know look hand heart hath left man allah...,Questions about moving forward/past,Police Brutality,Lekki Massacre


# Semi Supervised Multilabel

In [ ]:
import pandas as pd

In [ ]:
for_search= pd.read_csv('./sample_multi_label.csv')
for_search.head()

### Preprocessing

In [ ]:
%%capture
!pip install tweet-preprocessor
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.cluster import KMeans 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
import os,re
import preprocessor as p
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

In [ ]:
for_search.shape

(1134, 4)

In [ ]:
for_search['HumanLabel'].value_counts()

Irrelevant                             423
Activism                               407
Questions about moving forward/past     86
Lekki Massacre                          70
No Response from public figures         64
Police brutality events                 55
Media coverage                          21
Activism                                 2
 No Response from public figures         1
Police brutality events                  1
Name: HumanLabel, dtype: int64

In [ ]:
#Normalise all labels
for_search['HumanLabel'].replace(" No Response from public figures","No Response from public figures",inplace = True)
for_search['HumanLabel'].replace("Police brutality events ","Police brutality events",inplace = True)
for_search['HumanLabel'].replace("Activism ","Activism",inplace = True)

In [ ]:
for_search['HumanLabel'].value_counts()

Irrelevant                             423
Activism                               409
Questions about moving forward/past     86
Lekki Massacre                          70
No Response from public figures         65
Police brutality events                 56
Media coverage                          21
Name: HumanLabel, dtype: int64

In [ ]:
for_search.columns

Index(['tweet_id', 'tweet', 'cleaned_tweets', 'HumanLabel'], dtype='object')

In [ ]:
def remove_newline(message):
  return re.sub('\n', ' ', str(message.lower()))
def remove_symbols(message):
  re.sub(r'^\x00-\x7F+', ' ', message)
  return re.sub(r'[@!.,(\/&)?:#*...-;'']', '', str(message)) 
def remove_urls(message):
  return re.sub(r'http\S+', '', str(message))
# Remove emoji
from unicodedata import normalize
def remove_nonascii(message):
  unicode_norm = normalize('NFD', message).encode('ascii', 'ignore')
  return unicode_norm.decode('UTF-8')
 
def prep_message(message):
  message = remove_urls(message)
  message = remove_symbols(message)
  message = remove_newline(message)
  message = remove_nonascii(message)
  return message

In [ ]:
#Create your reference data
search_data = for_search[['tweet_id','cleaned_tweets','HumanLabel']]
search_data['cleaned_tweets']=[prep_message(word) for word in search_data['cleaned_tweets']]
search_data['cleaned_tweets']

In [ ]:
search_df = search_data.groupby('HumanLabel').agg({'cleaned_tweets': lambda x: ' '.join(x.astype(str))})
search_df

,cleaned_tweets
HumanLabel,
Activism,the government that's suppose to protect us is...
Irrelevant,thread and nawaaa for the of you omo if you us...
Lekki Massacre,what a country nigeria weeps tonight i will li...
Media coverage,twitter creates an emoji for the hashtag to su...
No Response from public figures,the uk government reacted with a video our own...
Police brutality events,they are blocked headed humanspls travel by ro...
Questions about moving forward/past,if it's not you or someone you care about toda...


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
search_df['tweet_without_stopwords'] = search_df['cleaned_tweets'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
search_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Put tokens for each label into groups
Activism = search_df['cleaned_tweets'].loc['Activism']
Irrelevant = search_df['cleaned_tweets'].loc['Irrelevant']
Lekki_Massacre = search_df['cleaned_tweets'].loc['Lekki Massacre']
No_Response_from_public_figures= search_df['cleaned_tweets'].loc['No Response from public figures']
Police_Brutality_Events=search_df['cleaned_tweets'].loc['Police brutality events']
Questions_about_moving_forward_past=search_df['cleaned_tweets'].loc['Questions about moving forward/past']
media_coverage=search_df['cleaned_tweets'].loc['Media coverage']

In [ ]:
Activism

'the government that\'s suppose to protect us is killing usto this end we\'ve unanimously say stop spreading rubbish focus the uk government reacted with a video our own elected president reacted with a tweet the height of irresponsibility our voice needs to be heard yesterdays survivors are todays victims and todays survivors will be tomorrows victims until we demand better from the people charged with the responsibility to protect us they are blocked headed humanspls travel by road through badagry express road to republic of benin you\'ll regret being nigerian these guyz are shameless they collect from motorists almost  checkpoints amp road blocks extorting transporters i believe u obo have spoken say no to privative discussion the buhari regime is a horrible situation words on marble our future in our hands please post all videos under this tweet am just a young nigerian with a bsc second class lower doing my hustling things just to meet my ends means u know while doing thisu don\'t

In [ ]:
#test data to be labelled
test_data = pd.read_csv('./test_data.csv')
y_test=test_data['HumanLabel']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#Importing & Initializing the "CountVectorizer" object, which is scikit-learn's bag of words tool. By default 'split()' will tokenize each tag using space.
def tokenize(x):
    x=x.split(',')
    tags=[i.strip() for i in x] #Some tags contains whitespaces before them, so we need to strip them
    return tags

# binary='true' will give a binary vectorizer
vectorizer = CountVectorizer(tokenizer = tokenize, binary='true').fit(y_test)
#y_train_multilabel = vectorizer.transform(y_train)
y_test_multilabel = vectorizer.transform(y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
y_test_multilabel

<245x7 sparse matrix of type '<class 'numpy.int64'>'
	with 322 stored elements in Compressed Sparse Row format>

In [ ]:
y_test_multilabel.shape

(245, 7)

### USE

In [ ]:
%%capture
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15

In [ ]:
import tensorflow as tf
from absl import logging
import tensorflow_hub as hub

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #"https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(module_url)

In [ ]:
tweets = test_data['cleaned_tweets'].to_list()

In [ ]:
def embed_tweets(messages):
  similarity_input_placeholder = tf.placeholder(tf.string, shape=([None]))
  similarity_message_encodings = embed(similarity_input_placeholder)
  with tf.Session() as session:
    session.run([tf.global_variables_initializer(),tf.tables_initializer()])
    message_embeddings_ = session.run(similarity_message_encodings, feed_dict={similarity_input_placeholder: messages})
  return message_embeddings_ 
  # run_and_plot(session, similarity_input_placeholder, messages,
  #              similarity_message_encodings)

In [ ]:
tweets_vector = embed_tweets(tweets)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
tweets_vector.shape

(245, 512)

In [ ]:
def embed_tweet(tweet):
  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed([tweet]))
  return message_embeddings

In [ ]:
Irrelevant_vector=embed_tweet(Irrelevant)
Irrelevant_vector.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(1, 512)

In [ ]:
Activism_vector=embed_tweet(Activism)
Activism_vector.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(1, 512)

In [ ]:
Activism_vector.shape

(1, 512)

In [ ]:
Lekki_Massacre_vector=embed_tweet(Lekki_Massacre)
Lekki_Massacre_vector.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(1, 512)

In [ ]:
No_Response_vector=embed_tweet(No_Response_from_public_figures)
No_Response_vector.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(1, 512)

In [ ]:
Police_Brutality_vector=embed_tweet(Police_Brutality_Events)
Police_Brutality_vector.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(1, 512)

In [ ]:
Questions_vector=embed_tweet(Questions_about_moving_forward_past)
Questions_vector.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(1, 512)

In [ ]:
media_coverage_vector=embed_tweet(media_coverage)
media_coverage_vector.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(1, 512)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
def get_cosine_scores(group,tweets):
  cosine_similarities = linear_kernel(group,tweets).flatten()
  document_scores = [item.item() for item in cosine_similarities[:]]
  return document_scores

In [ ]:
Irrelevant_cosine_scores = get_cosine_scores(Irrelevant_vector,tweets_vector)
Activism_cosine_scores = get_cosine_scores(Activism_vector,tweets_vector)
LekkiMassacre_cosine_scores = get_cosine_scores(Lekki_Massacre_vector,tweets_vector)
NoResponse_cosine_scores = get_cosine_scores(No_Response_vector,tweets_vector)
PoliceBrutality_cosine_scores = get_cosine_scores(Police_Brutality_vector,tweets_vector)
Questions_cosine_scores = get_cosine_scores(Questions_vector,tweets_vector)
media_cosine_scores = get_cosine_scores(media_coverage_vector,tweets_vector)

[0.13396453857421875,
 0.339614599943161,
 0.3846173584461212,
 0.36717742681503296,
 0.22675715386867523,
 0.45463913679122925,
 0.4654589295387268,
 0.3977149724960327,
 0.406175822019577,
 0.3247862458229065]

In [ ]:
cosine_data  = {'tweet_id':test_data['tweet_id'].to_list(),'Original_Tweets':test_data['tweet'].to_list(),'Cleaned_Tweets':test_data['cleaned_tweets'].to_list(),'Human_label':test_data['HumanLabel'].to_list(), 'Activism':Activism_cosine_scores,
         'Lekki Massacre': LekkiMassacre_cosine_scores, 'No Response from public figures':NoResponse_cosine_scores, 
         'Police brutality events':PoliceBrutality_cosine_scores,'Questions about moving forward/past':Questions_cosine_scores,'Media coverage':media_cosine_scores,'Irrelevant':Irrelevant_cosine_scores}
USEscores_df = pd.DataFrame(cosine_data)
USEscores_df.head(2)

In [ ]:
USEscores_df.dropna(subset=['Cleaned_Tweets'], inplace=True)
top3_USE = pd.DataFrame(USEscores_df.iloc[:,4:].columns.values[np.argsort(-USEscores_df.iloc[:,4:].values, axis=1)[:, :3]],
                  columns = ['1st Max','2nd Max','3rd Max'])

(245, 11)

In [ ]:
top3_test_USE = pd.DataFrame(USEscores_df.iloc[:,4:].columns.values[np.argsort(-USEscores_df.iloc[:,4:].values, axis=1)[:, :3]],
                  columns = ['1st Max','2nd Max','3rd Max'])

In [ ]:
#Put it all together
USEscores_df['Predicted label'] = top3_test_USE[['1st Max','2nd Max','3rd Max']].apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)

In [ ]:
y_USE = USEscores_df['Predicted label']
y_USE_multilabel = vectorizer.transform(y_USE)

In [ ]:
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score

In [ ]:
print("Accuracy :",metrics.accuracy_score(y_test_multilabel, y_USE_multilabel))
print("Hamming loss ",metrics.hamming_loss(y_test_multilabel,y_USE_multilabel))

precision = precision_score(y_test_multilabel, y_USE_multilabel, average='micro')
recall = recall_score(y_test_multilabel, y_USE_multilabel, average='micro')
f1 = f1_score(y_test_multilabel, y_USE_multilabel, average='micro')
 
print("\nMicro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test_multilabel, y_USE_multilabel, average='macro')
recall = recall_score(y_test_multilabel, y_USE_multilabel, average='macro')
f1 = f1_score(y_test_multilabel, y_USE_multilabel, average='macro')
 
print("\nMacro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print("\nClassification Report")
print (metrics.classification_report(y_test_multilabel, y_USE_multilabel))
# print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0
Hamming loss  0.39708454810495625

Micro-average quality numbers
Precision: 0.2558, Recall: 0.5839, F1-measure: 0.3557

Macro-average quality numbers
Precision: 0.2592, Recall: 0.6630, F1-measure: 0.3240

Classification Report
              precision    recall  f1-score   support

           0       0.68      0.52      0.59       154
           1       0.35      0.55      0.43        53
           2       0.14      0.90      0.25        21
           3       0.15      0.72      0.25        18
           4       0.10      0.81      0.17        16
           5       0.24      0.59      0.34        29
           6       0.15      0.55      0.24        31

   micro avg       0.26      0.58      0.36       322
   macro avg       0.26      0.66      0.32       322
weighted avg       0.44      0.58      0.44       322
 samples avg       0.26      0.60      0.35       322



In [ ]:
USE_labels  = {'tweet_id':test_data['tweet_id'].to_list(),'Original_Tweets':test_data['tweet'].to_list(),'Cleaned_Tweets':test_data['cleaned_tweets'].to_list(),'Human_label':test_data['HumanLabel'].to_list(),'USE_label':USEscores_df['Predicted label']}
USE_labels_df = pd.DataFrame(USE_labels)
USE_labels_df.head()

,tweet_id,Original_Tweets,Cleaned_Tweets,Human_label,USE_label
0,1.321870e+18,@lshaofficial @DesmondOElliot @DesmondElliot_ ...,the change begins with you yes you,Activism,"Activism, No Response from public figures, Que..."
1,1.315650e+18,"I don't think there is any curse in Elvish, En...",i don't think there is any curse in elvish ent...,Irrelevant,"No Response from public figures, Lekki Massacr..."
2,1.318140e+18,The Egyptians we see today we shall see them n...,the egyptians we see today we shall see them n...,Activism,"Lekki Massacre, Activism, No Response from pub..."
3,1.317260e+18,SARS!!!! The current Nigerian government!!!\n#...,sars the current nigerian government,Activism,"No Response from public figures, Police brutal..."
4,1.315270e+18,@Olamide Sure we deserve #EndSarsNow,sure we deserve,Activism,"Activism, Lekki Massacre, Questions about movi..."


### TFIDF 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 

# Uncomment here to download the latest nltk stop words
import nltk
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

class LemmaTokenizer:
    """
    Interface to the WordNet lemmatizer from nltk
    """
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]

tokenizer=LemmaTokenizer()

# Lemmatize the stop words
stop_words = tokenizer(' '.join(stop_words))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Create TF-idf model
vectorizer = TfidfVectorizer(stop_words=stop_words, 
                               tokenizer=tokenizer)

In [ ]:
media_coverage

"twitter creates an emoji for the hashtag to support the fight of nigrians against corruption and police violence the lagos state governor is live on arise news telling us lies again channels tv arise tv tvc ait especially i am blocking all stations on my and social media no need watching or seeing your message again you are useless abuja protesters behind cbn are been attacked rn theyve tried calling the helpline but they arent picking please rt and tag the right people the leaders are running out of the state the aba peaceful protest didn't end well as many innocent citizens was killed by the amp the citizens have to defend themselves and chase the away and the whole aba was on fire many police stations was brunt more videos comingup lagos state governor imposes -hour curfew on all parts of lagos starting pm today citing security concerns tonight on newz lookout for my report on the calls by nigerians living in sa for their gov to set up an independent body to probe all cases of poli

In [ ]:
Search_vector=vectorizer.fit_transform([Activism]+[Lekki_Massacre]+[No_Response_from_public_figures]+[Police_Brutality_Events]+[Questions_about_moving_forward_past]+[media_coverage]+[Irrelevant])
Search_vector

<7x3262 sparse matrix of type '<class 'numpy.float64'>'
	with 5852 stored elements in Compressed Sparse Row format>

In [ ]:
test_data.head()

,tweet_id,tweet,cleaned_tweets,HumanLabel
0,1.321870e+18,@lshaofficial @DesmondOElliot @DesmondElliot_ ...,the change begins with you yes you,Activism
1,1.315650e+18,"I don't think there is any curse in Elvish, En...",i don't think there is any curse in elvish ent...,Irrelevant
2,1.318140e+18,The Egyptians we see today we shall see them n...,the egyptians we see today we shall see them n...,Activism
3,1.317260e+18,SARS!!!! The current Nigerian government!!!\n#...,sars the current nigerian government,Activism
4,1.315270e+18,@Olamide Sure we deserve #EndSarsNow,sure we deserve,Activism


In [ ]:
test_vectors=vectorizer.transform(test_data['cleaned_tweets'])
test_vectors

<245x3262 sparse matrix of type '<class 'numpy.float64'>'
	with 1535 stored elements in Compressed Sparse Row format>

In [ ]:
Activism_vectors=Search_vector[0:1]
Lekki_Massacre_vectors = Search_vector[1:2]
No_Response_vectors = Search_vector[2:3]
Police_vectors = Search_vector[3:4]
Questions_vectors = Search_vector[4:5]
media_vectors = Search_vector[5:6]
Irrelevant_vectors=Search_vector[6:7]

#### Collation pipeline tfidf

In [ ]:
def get_cosine_scores(group,tweets):
  cosine_similarities = linear_kernel(group,tweets).flatten()
  document_scores = [item.item() for item in cosine_similarities[:]]
  return document_scores

In [ ]:
Activism_tfidf_cosine_scores = get_cosine_scores(Activism_vectors,test_vectors)
Irrelevant_tfidf_cosine_scores = get_cosine_scores(Irrelevant_vectors,test_vectors)
LekkiMassacre_tfidf_cosine_scores = get_cosine_scores(Lekki_Massacre_vectors,test_vectors)
NoResponse_tfidf_cosine_scores = get_cosine_scores(No_Response_vectors,test_vectors)
PoliceBrutality_tfidf_cosine_scores = get_cosine_scores(Police_vectors,test_vectors)
Questions_tfidf_cosine_scores = get_cosine_scores(Questions_vectors,test_vectors)
media_tfidf_cosine_scores = get_cosine_scores(media_vectors,test_vectors)

[0.023236450618662564,
 0.11112198541798703,
 0.11221254694732274,
 0.16848647918127532,
 0.024893432386764898,
 0.06738090916144121,
 0.14310925570027322,
 0.07429934308257917,
 0.05772935790297352,
 0.18485719787890625]

In [ ]:
test_data.head()

,tweet_id,tweet,cleaned_tweets,HumanLabel
0,1.321870e+18,@lshaofficial @DesmondOElliot @DesmondElliot_ ...,the change begins with you yes you,Activism
1,1.315650e+18,"I don't think there is any curse in Elvish, En...",i don't think there is any curse in elvish ent...,Irrelevant
2,1.318140e+18,The Egyptians we see today we shall see them n...,the egyptians we see today we shall see them n...,Activism
3,1.317260e+18,SARS!!!! The current Nigerian government!!!\n#...,sars the current nigerian government,Activism
4,1.315270e+18,@Olamide Sure we deserve #EndSarsNow,sure we deserve,Activism


In [ ]:
'''new df with names, and the cosine scores for each group'''
tf_cosine_data  = {'tweet_id':test_data['tweet_id'].to_list(),'Original_Tweets':test_data['tweet'].to_list(),'Cleaned_Tweets':test_data['cleaned_tweets'].to_list(),'Human_label':test_data['HumanLabel'].to_list(), 'Activism':Activism_tfidf_cosine_scores,
         'Lekki Massacre': LekkiMassacre_tfidf_cosine_scores, 'No Response from public figures':NoResponse_tfidf_cosine_scores, 
         'Police brutality events':PoliceBrutality_tfidf_cosine_scores,'Questions about moving forward/past':Questions_tfidf_cosine_scores,'Media coverage':media_tfidf_cosine_scores,'Irrelevant':Irrelevant_tfidf_cosine_scores}
tfidf_scores_df = pd.DataFrame(tf_cosine_data)
tfidf_scores_df.head(2)
tfidf_scores_df['Cleaned_Tweets'].replace('', np.nan, inplace=True)
tfidf_scores_df

,tweet_id,Original_Tweets,Cleaned_Tweets,Human_label,Activism,Lekki Massacre,No Response from public figures,Police brutality events,Questions about moving forward/past,Media coverage,Irrelevant
0,1.321870e+18,@lshaofficial @DesmondOElliot @DesmondElliot_ ...,the change begins with you yes you,Activism,0.041507,0.000000,0.011123,0.028024,0.043580,0.000000,0.028493
1,1.315650e+18,"I don't think there is any curse in Elvish, En...",i don't think there is any curse in elvish ent...,Irrelevant,0.113019,0.108349,0.124470,0.048902,0.127308,0.025844,0.109595


In [ ]:
tfidf_scores_df.dropna(subset=['Cleaned_Tweets'], inplace=True)
tfidf_scores_df.shape
top3_test_tfidf = pd.DataFrame(tfidf_scores_df.iloc[:,4:].columns.values[np.argsort(-tfidf_scores_df.iloc[:,4:].values, axis=1)[:, :3]],
                  columns = ['1st Max','2nd Max','3rd Max'])
top3_test_tfidf
tfidf_scores_df['Predicted label'] = top3_test_tfidf[['1st Max','2nd Max','3rd Max']].apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)
tfidf_scores_df['Predicted label']
tfidf_labels  = {'tweet_id':test_data['tweet_id'].to_list(),'Original_Tweets':test_data['tweet'].to_list(),'Cleaned_Tweets':test_data['cleaned_tweets'].to_list(),'Human_label':test_data['HumanLabel'].to_list(),'tfidf_label':tfidf_scores_df['Predicted label']}
tfidf_labels_df = pd.DataFrame(tfidf_labels)
tfidf_labels_df.head()

(245, 11)

In [ ]:
y_test_multilabel

<245x7 sparse matrix of type '<class 'numpy.int64'>'
	with 322 stored elements in Compressed Sparse Row format>

In [ ]:
y_tfidf = tfidf_scores_df['Predicted label'] 
#y_tfidf_multilabel = vectorizer.transform(y_tfidf)
vectorizer = CountVectorizer(tokenizer = tokenize, binary='true').fit(y_test)
#y_train_multilabel = vectorizer.transform(y_train)
y_tfidf_multilabel = vectorizer.transform(y_tfidf)

In [ ]:
print("Accuracy :",metrics.accuracy_score(y_test_multilabel, y_tfidf_multilabel))
print("Hamming loss ",metrics.hamming_loss(y_test_multilabel,y_tfidf_multilabel))

precision = precision_score(y_test_multilabel, y_tfidf_multilabel, average='micro')
recall = recall_score(y_test_multilabel, y_tfidf_multilabel, average='micro')
f1 = f1_score(y_test_multilabel, y_tfidf_multilabel, average='micro')
 
print("\nMicro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test_multilabel, y_tfidf_multilabel, average='macro')
recall = recall_score(y_test_multilabel, y_tfidf_multilabel, average='macro')
f1 = f1_score(y_test_multilabel, y_tfidf_multilabel, average='macro')
 
print("\nMacro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print("\nClassification Report")
print (metrics.classification_report(y_test_multilabel, y_tfidf_multilabel))
# print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0
Hamming loss  0.3492711370262391

Micro-average quality numbers
Precision: 0.3116, Recall: 0.7112, F1-measure: 0.4333

Macro-average quality numbers
Precision: 0.2430, Recall: 0.6178, F1-measure: 0.3285

Classification Report
              precision    recall  f1-score   support

           0       0.67      0.84      0.75       154
           1       0.22      0.64      0.33        53
           2       0.19      0.76      0.30        21
           3       0.12      0.17      0.14        18
           4       0.15      0.81      0.25        16
           5       0.18      0.52      0.27        29
           6       0.16      0.58      0.25        31

   micro avg       0.31      0.71      0.43       322
   macro avg       0.24      0.62      0.33       322
weighted avg       0.42      0.71      0.50       322
 samples avg       0.31      0.73      0.42       322



### Jaccard similarity

In [ ]:
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

def get_scores(group,tweets):
    scores = []
    for tweet in tweets:
        s = jaccard_similarity(group, tweet)
        scores.append(s)
    return scores

In [ ]:
Activism_jaccard_scores = get_scores(Activism, test_data['cleaned_tweets'].to_list())
Irrelevant_jaccard_scores = get_scores(Irrelevant,test_data['cleaned_tweets'].to_list())
Massacre_jaccard_scores = get_scores(Lekki_Massacre, test_data['cleaned_tweets'].to_list())
No_Response_jaccard_scores = get_scores(No_Response_from_public_figures, test_data['cleaned_tweets'].to_list())
Police_Brutality_jaccard_scores = get_scores(Police_Brutality_Events, test_data['cleaned_tweets'].to_list())
Questions_jaccard_scores = get_scores(Questions_about_moving_forward_past, test_data['cleaned_tweets'].to_list())
media_jaccard_scores = get_scores(media_coverage, test_data['cleaned_tweets'].to_list())

[0.4117647058823529,
 0.7058823529411765,
 0.6176470588235294,
 0.6470588235294118,
 0.5,
 0.7058823529411765,
 0.5588235294117647,
 0.6176470588235294,
 0.7058823529411765,
 0.5588235294117647]

In [ ]:
jaccard_test  = {'tweet_id':test_data['tweet_id'].to_list(),'Original_Tweets':test_data['tweet'].to_list(),'Cleaned_Tweets':test_data['cleaned_tweets'].to_list(),'Human_label':test_data['HumanLabel'].to_list(),'Activism':Activism_jaccard_scores,'Lekki Massacre':Massacre_jaccard_scores, 'No Response from public figures':No_Response_jaccard_scores,'Police brutality events':Police_Brutality_jaccard_scores,'Questions about moving forward/past':Questions_jaccard_scores,'Media coverage':media_jaccard_scores,'Irrelevant':Irrelevant_jaccard_scores}
jaccard_test = pd.DataFrame(jaccard_test)
jaccard_test.head(20)

In [ ]:
top_test_jaccard

,1st Max,2nd Max,3rd Max
0,Lekki Massacre,Media coverage,Questions about moving forward/past
1,Lekki Massacre,Media coverage,Questions about moving forward/past
2,Lekki Massacre,Media coverage,Questions about moving forward/past
3,Lekki Massacre,Media coverage,Questions about moving forward/past
4,Lekki Massacre,Media coverage,Questions about moving forward/past
...,...,...,...
240,Lekki Massacre,Questions about moving forward/past,Media coverage
241,Lekki Massacre,Media coverage,Questions about moving forward/past
242,Lekki Massacre,Media coverage,Questions about moving forward/past
243,Lekki Massacre,Media coverage,Questions about moving forward/past


In [ ]:
jaccard_test['Predicted label'] = top_test_jaccard[['1st Max','2nd Max','3rd Max']].apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)
jaccard_test.head(15)

In [ ]:
y_jaccard = jaccard_test['Predicted label']

In [ ]:
#vectorizer = CountVectorizer(tokenizer = tokenize, binary='true').fit(y_test)
#y_train_multilabel = vectorizer.transform(y_train)
y_jaccard_multilabel = vectorizer.transform(y_jaccard)

In [ ]:
y_jaccard_multilabel

<245x7 sparse matrix of type '<class 'numpy.int64'>'
	with 735 stored elements in Compressed Sparse Row format>

In [ ]:
print("Accuracy :",metrics.accuracy_score(y_test_multilabel, y_jaccard_multilabel))
print("Hamming loss ",metrics.hamming_loss(y_test_multilabel,y_jaccard_multilabel))

precision = precision_score(y_test_multilabel, y_jaccard_multilabel, average='micro')
recall = recall_score(y_test_multilabel, y_jaccard_multilabel, average='micro')
f1 = f1_score(y_test_multilabel, y_jaccard_multilabel, average='micro')
 
print("\nMicro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test_multilabel, y_jaccard_multilabel, average='macro')
recall = recall_score(y_test_multilabel, y_jaccard_multilabel, average='macro')
f1 = f1_score(y_test_multilabel, y_jaccard_multilabel, average='macro')
 
print("\nMacro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print("\nClassification Report")
print (metrics.classification_report(y_test_multilabel, y_jaccard_multilabel))
# print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.0
Hamming loss  0.5346938775510204

Micro-average quality numbers
Precision: 0.0952, Recall: 0.2174, F1-measure: 0.1325

Macro-average quality numbers
Precision: 0.0410, Recall: 0.4286, F1-measure: 0.0745

Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       154
           1       0.00      0.00      0.00        53
           2       0.09      1.00      0.16        21
           3       0.07      1.00      0.14        18
           4       0.00      0.00      0.00        16
           5       0.00      0.00      0.00        29
           6       0.13      1.00      0.23        31

   micro avg       0.10      0.22      0.13       322
   macro avg       0.04      0.43      0.07       322
weighted avg       0.02      0.22      0.04       322
 samples avg       0.10      0.17      0.12       322



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
final_test_jaccard=pd.concat([jaccard_test.iloc[:,:4],jaccard_test['Predicted label']], axis=1, ignore_index=False)
final_test_jaccard

In [ ]:
all_labels  = {'tweet_id':test_data['tweet_id'].to_list(),'Original_Tweets':test_data['tweet'].to_list(),'Cleaned_Tweets':test_data['cleaned_tweets'].to_list(),'Human_label':test_data['HumanLabel'].to_list(),'tfidf_cosine_label':tfidf_scores_df['Predicted label'],'USE_cosine_label':USEscores_df['Predicted label'],'jaccard_label':jaccard_test['Predicted label']}
all_labels_df = pd.DataFrame(all_labels)
all_labels_df.head(20)

In [ ]:
all_labels_df['cosine_tfidf_human'] = np.where((all_labels_df['tfidf_cosine_label'] == all_labels_df['Human_label']),1, np.nan)
all_labels_df['cosine_USE_human'] = np.where((all_labels_df['USE_cosine_label'] == all_labels_df['Human_label']),1, np.nan)
all_labels_df['jaccard_human'] = np.where((all_labels_df['jaccard_label'] == all_labels_df['Human_label']),1, np.nan)


In [ ]:
all_labels_df.shape

(245, 10)

In [ ]:
jaccard_tfidf_human = (all_labels_df['jaccard_human'].value_counts()/246)*100
jaccard_tfidf_human

Series([], Name: jaccard_human, dtype: float64)

In [ ]:
cosine_tfidf_human = (all_labels_df['cosine_tfidf_human'].value_counts()/246)*100
cosine_tfidf_human

Series([], Name: cosine_tfidf_human, dtype: float64)

In [ ]:
cosine_USE_human = (all_labels_df['cosine_USE_human'].value_counts()/246)*100
cosine_USE_human

Series([], Name: cosine_USE_human, dtype: float64)

In [ ]:
# from sklearn.metrics.pairwise import linear_kernel

In [ ]:
# def get_cosine_scores(group,tweets):
#   cosine_similarities = linear_kernel(group,tweets).flatten()
#   document_scores = [item.item() for item in cosine_similarities[:]]
#   return document_scores

# Zero-Shot Text Classification with huggingface

In [ ]:
!pip install transformers
from transformers import pipeline

In [ ]:
from tqdm import tqdm

In [ ]:
df = pd.read_csv('Your_data.csv')
classifier = pipeline("zero-shot-classification",device=0) #download the pretrained model - sr device = 0 to use GPU
candidate_labels = ['Media coverage','Activism','Police brutality events','No response from public figures','Questions about moving forward/past','Lekki Massacre','Irrelevant']

In [ ]:
predictedCategories = []
#categories_scores = []
tweet_scores = []
for i in tqdm(range(len(df))):
    text = df.iloc[i,]['cleaned_tweets']
    res = classifier(text, candidate_labels, multi_class=True) #setting multi-class as True gives multilabel 
    labels = res['labels'] 
    scores = res['scores'] #extracting the scores associated with the labels
    res_dict = {label : score for label,score in zip(labels, scores)}
    sorted_dict = dict(sorted(res_dict.items(), key=lambda x:x[0],reverse = False)) #sorting the dictionary of labels in alphabetical order based on their score
    t_score  = []
    #categories_score = []
    for i, (k,v) in enumerate(sorted_dict.items()):
      #categories_score.append(k)
      t_score.append(v)
    tweet_scores.append(t_score) #return all scores with categories sorted alphabetical order
   # categories_scores.append(categories_score)
    categories  = []
    sorted_dict_ = dict(sorted(res_dict.items(), key=lambda x:x[1],reverse = True))
    for i, (k,v) in enumerate(sorted_dict_.items()):
      if(i > 2): #return categories with only the best 3 predictions
        break
      else:
        categories.append(k)
    predictedCategories.append(categories)

  4%|▍         | 10/245 [00:01<00:37,  6.26it/s]/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:901: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
100%|██████████| 245/245 [00:40<00:00,  6.01it/s]


In [ ]:
#Put it all together
tweet_scores_df = pd.DataFrame(tweet_scores, columns=['Activism','Lekki Massacre','Media coverage','No response from public figures','Police Brutality','Questions about moving forward/past','Irrelevant'])
tweet_scores_df

,Activism,Lekki Massacre,Media coverage,No response from public figures,Police Brutality,Questions about moving forward/past,Irrelevant
0,0.754780,0.000148,0.012566,0.122678,0.000159,0.039525,0.396220
1,0.025495,0.001117,0.048469,0.018139,0.003023,0.047655,0.189236
2,0.014667,0.003151,0.006352,0.035611,0.001678,0.000117,0.949163
3,0.011175,0.000397,0.013504,0.004388,0.000089,0.000939,0.247348
4,0.289728,0.000376,0.034569,0.339679,0.000332,0.089706,0.334366
...,...,...,...,...,...,...,...
240,0.121981,0.000145,0.004028,0.012895,0.000226,0.000634,0.660356
241,0.959354,0.000257,0.034022,0.039049,0.000395,0.214194,0.383004
242,0.249530,0.000489,0.011538,0.249910,0.019596,0.928222,0.675362
243,0.145861,0.000235,0.000386,0.061604,0.000102,0.578648,0.370735


In [ ]:
#Select the  top-3 most confident predictions
tweet_label_df = pd.DataFrame(predictedCategories, columns=['Label 1','Label 2','Label 3'])
tweet_label_df

,Label 1,Label 2,Label 3
0,Activism,Questions about moving forward/past,Media coverage
1,Questions about moving forward/past,Lekki Massacre,Police brutality events
2,Questions about moving forward/past,Media coverage,Activism
3,Questions about moving forward/past,Lekki Massacre,Activism
4,Media coverage,Questions about moving forward/past,Activism
...,...,...,...
240,Questions about moving forward/past,Activism,Media coverage
241,Activism,Questions about moving forward/past,Police brutality events
242,Police brutality events,Questions about moving forward/past,Media coverage
243,Police brutality events,Questions about moving forward/past,Activism


In [ ]:
#Combine all labels
tweet_label_df['Predicted label'] = tweet_label_df[['Label 1','Label 2','Label 3']].apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)
y_ZeroShot = tweet_label_df['Predicted label']
y_ZeroShot_multilabel = vectorizer.transform(y_ZeroShot)
y_ZeroShot_multilabel

<245x7 sparse matrix of type '<class 'numpy.int64'>'
	with 735 stored elements in Compressed Sparse Row format>

In [ ]:
print("Accuracy :",metrics.accuracy_score(y_test_multilabel, y_ZeroShot_multilabel))
print("Hamming loss ",metrics.hamming_loss(y_test_multilabel,y_ZeroShot_multilabel))

precision = precision_score(y_test_multilabel, y_ZeroShot_multilabel, average='micro')
recall = recall_score(y_test_multilabel, y_ZeroShot_multilabel, average='micro')
f1 = f1_score(y_test_multilabel, y_ZeroShot_multilabel, average='micro')
 
print("\nMicro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test_multilabel, y_ZeroShot_multilabel, average='macro')
recall = recall_score(y_test_multilabel, y_ZeroShot_multilabel, average='macro')
f1 = f1_score(y_test_multilabel, y_ZeroShot_multilabel, average='macro')
 
print("\nMacro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print("\nClassification Report")
print (metrics.classification_report(y_test_multilabel, y_ZeroShot_multilabel))
# print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.02040816326530612
Hamming loss  0.3434402332361516

Micro-average quality numbers
Precision: 0.3184, Recall: 0.7267, F1-measure: 0.4428

Macro-average quality numbers
Precision: 0.3698, Recall: 0.6607, F1-measure: 0.3492

Classification Report
              precision    recall  f1-score   support

           0       0.68      0.92      0.78       154
           1       0.80      0.08      0.14        53
           2       0.30      0.76      0.43        21
           3       0.12      0.89      0.21        18
           4       0.31      0.25      0.28        16
           5       0.25      0.76      0.38        29
           6       0.13      0.97      0.22        31

   micro avg       0.32      0.73      0.44       322
   macro avg       0.37      0.66      0.35       322
weighted avg       0.53      0.73      0.51       322
 samples avg       0.32      0.70      0.42       322



In [ ]:
tweet_data_plus_labels  = {'tweet_id':test_data['tweet_id'].to_list(),'Original_Tweets':test_data['tweet'].to_list(),'Cleaned_Tweets':test_data['cleaned_tweets'].to_list(),'Human_label':test_data['HumanLabel'].to_list(), 'Zero_shot_label':tweet_label_df['Predicted label'].to_list()}
Zero_shot_df = pd.DataFrame(tweet_data_plus_labels,pd.set_option('display.max_colwidth', -1))
Zero_shot_df.head(2)

In [ ]:
top3_USE

,1st Max,2nd Max,3rd Max
0,Activism,No Response from public figures,Questions about moving forward/past
1,No Response from public figures,Lekki Massacre,Irrelevant
2,Lekki Massacre,Activism,No Response from public figures
3,No Response from public figures,Police brutality events,Media coverage
4,Activism,Lekki Massacre,Questions about moving forward/past
...,...,...,...
240,No Response from public figures,Irrelevant,Police brutality events
241,Activism,Lekki Massacre,Media coverage
242,No Response from public figures,Irrelevant,Police brutality events
243,Media coverage,Lekki Massacre,Questions about moving forward/past


#Evaluating

In [ ]:
all_labels  = {'tweet_id':test_data['tweet_id'].to_list(),'Original_Tweets':test_data['tweet'].to_list(),'Cleaned_Tweets':test_data['cleaned_tweets'].to_list(),'Human_label':test_data['HumanLabel'].to_list(),'USE_label':USE_labels_df['USE_label'].to_list(),'tfidf_label':tfidf_labels_df['tfidf_label'].to_list(),'jaccard_label':jaccard_test['Predicted label'].to_list(),'Zero_shot_label':Zero_shot_df['Zero_shot_label'].to_list(),'humanvsZero':Zero_shot_df['zero_shot_human'].to_list()}
all_labels_df = pd.DataFrame(all_labels)
all_labels_df.head(20)

In [ ]:
all_labels_df['cosine_tfidf_human'] = np.where((all_labels_df['tfidf_label'] == all_labels_df['Human_label']),1, np.nan)
all_labels_df['cosine_USE_human'] = np.where((all_labels_df['USE_label'] == all_labels_df['Human_label']),1, np.nan)
all_labels_df['jaccard_human'] = np.where((all_labels_df['jaccard_label'] == all_labels_df['Human_label']),1, np.nan)
all_labels_df['zero_shot_human'] = np.where((all_labels_df['Zero_shot_label'] == all_labels_df['Human_label']),1, np.nan)

In [ ]:
all_labels_df.shape

(245, 13)

In [ ]:
cosine_tfidf_human = (all_labels_df['cosine_tfidf_human'].value_counts()/246)*100
cosine_tfidf_human

1.0    34.95935
Name: cosine_tfidf_human, dtype: float64

In [ ]:
cosine_USE_human = (all_labels_df['cosine_USE_human'].value_counts()/246)*100
cosine_USE_human

1.0    30.487805
Name: cosine_USE_human, dtype: float64

In [ ]:
Zero_shot_human = (Zero_shot_df['zero_shot_human'].value_counts()/246)*100
Zero_shot_human

1.0    37.804878
Name: zero_shot_human, dtype: float64